In [3]:
pip install torchtext

Defaulting to user installation because normal site-packages is not writeable
  Using cached torch-1.8.1-cp38-cp38-manylinux1_x86_64.whl (804.1 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0
    Uninstalling torch-1.13.0:
      Successfully uninstalled torch-1.13.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: Could not install packages due to an OSError: [Errno 39] Directory not empty: 'lib'

You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
import pandas as pd
import numpy as np
import os
import re
import torch.nn as nn
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
import spacy
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error
from nltk.corpus import stopwords
from nltk import word_tokenize
import torchtext
from torchtext.data import get_tokenizer

ImportError: /usr/local/lib/python3.8/site-packages/torchtext/_torchtext.so: undefined symbol: _ZNK3c104Type14isSubtypeOfExtERKSt10shared_ptrIS0_EPSo

In [ ]:
data_dir = "/project/ds7003-fall22/team1/data/abstracts_data/200k_abstracts/"

In [ ]:
filenames = [data_dir + filename for filename in os.listdir(data_dir)]
filenames

In [ ]:
def get_lines(filename):
    with open(filename, 'r') as f:
        return f.readlines()

In [ ]:
train_data_lines = get_lines(data_dir+'train.txt')
train_data_lines[:20]

In [ ]:
def preprocess_data(filename):
    input_lines = get_lines(filename)
    abstract_lines = ""
    abstract_samples = []
    for line in input_lines:
        if line.startswith("###"):
            abstract_id = line
            abstract_lines = ""
        elif line.isspace():
            abstract_line_split = abstract_lines.splitlines()
            for abstract_line_number, abstract_line in enumerate(abstract_line_split):
                line_data={}
                target_text_split=abstract_line.split("\t")
                line_data["target"]=target_text_split[0]
                line_data["text"]=target_text_split[1].lower()
                line_data["line_number"] = abstract_line_number
                line_data["total_lines"] = len(abstract_line_split)-1
                abstract_samples.append(line_data)
        else:
            abstract_lines+=line
    return abstract_samples

In [ ]:
train_samples = preprocess_data(data_dir + "train.txt")
test_samples = preprocess_data(data_dir + "test.txt")

In [1]:
len(train_samples), len(test_samples)

NameError: name 'train_samples' is not defined

In [ ]:
train_samples[:20]

In [ ]:
train_df = pd.DataFrame(train_samples)
test_df = pd.DataFrame(test_samples)

In [ ]:
train_df.head(11)

In [ ]:
train_df.target.value_counts()

In [ ]:
print(train_df.shape)

In [ ]:
train_df = train_df.reset_index(drop=True)

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = BAD_SYMBOLS_RE.sub('', text)
    text = text.replace('x', '')
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

train_df["text"] = train_df["text"].apply(clean_text)

In [ ]:
test_df["text"] = test_df["text"].apply(clean_text)

In [ ]:
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer("basic_english")

def build_vocabulary(datasets):
    for dataset in datasets:
        for _, text in dataset:
            yield tokenizer(text)
            
vocab = build_vocab_from_iterator(build_vocabulary([train_df, test_df]), min_freq = 1, specials=["<UNK>"])
vocab.set_default_index(vocab["<UNK>"])